Importando bibliotecas

In [1]:
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import os

Carregando variáveis de ambiente, como a YOUTUBE_API_KEY

In [2]:
%load_ext dotenv
%dotenv
load_dotenv()


True

Definindo parâmetros para as chamadas de API

In [3]:
api_url = 'https://www.googleapis.com/youtube/v3/playlistItems'
metrics_url = 'https://www.googleapis.com/youtube/v3/videos'
youtube_api_key = os.environ.get('YOUTUBE_API_KEY')
playlist_id = 'PLpbom12S-UaJEDmUaFfWLws317OUKNceE'

part1 = 'snippet'
part2 = 'status'
part3 = 'statistics'
max_results = 50

Função: retornar resultado da API do YouTube

In [4]:
def use_requests(api_url):

    videos = []

    proceed = True
    page_token = ''

    while(proceed):

        res = requests.get(f'{api_url}?key={youtube_api_key}&part={part1}&part={part2}&playlistId={playlist_id}&maxResults={max_results}&pageToken={page_token}')

        json_res = json.loads(res.text)

        if 'nextPageToken' in json_res:
            page_token = json_res['nextPageToken']
        else:
            proceed = False

        videos += json_res['items']

    for v in videos:
        if v['status']['privacyStatus'] != 'public':
            videos.remove(v)
    
    return videos

Função: extrair estatísticas por meio de outra chamada de API

In [5]:
def get_metrics(metrics_url, videos):

    for v in videos:
        id = v['snippet']['resourceId']['videoId']
        res = requests.get(f'{metrics_url}?key={youtube_api_key}&part={part3}&id={id}')

        json_res = json.loads(res.text)

        v['statistics'] = json_res['items'][0]['statistics']

    return videos

Chamadas das funções para produzir os objetos

In [6]:
videos = use_requests(api_url) # retorna o snippet, como título, publicação etc

videos_metrics = get_metrics(metrics_url, videos) # adiciona as métricas, como likes e comentários

Definindo um novo objeto com as métricas a serem utilizadas: título, visualizações, likes e comentários

In [7]:
videos_df = []

for v in videos_metrics:

    v_df = {}
    v_df['title'] = v['snippet']['title']
    v_df['viewCount'] = v['statistics']['viewCount']
    v_df['likeCount'] = v['statistics']['likeCount']
    v_df['commentCount'] = v['statistics']['commentCount']

    if v['status']['privacyStatus'] == 'public':
        videos_df.append(v_df)

Resultado final

In [8]:
print(json.dumps(videos_df, indent=4))

[
    {
        "title": "Fundos de CRIs (KNCR11, KNIP11, KNHY11, KNSC11 e KCRE11) - Resumo Mensal ref. Julho.22",
        "viewCount": "26780",
        "likeCount": "247",
        "commentCount": "9"
    },
    {
        "title": "Fundo de Infraestrutura KDIF11 - Resumo Mensal do Kinea Infra ref. Julho.22",
        "viewCount": "26060",
        "likeCount": "254",
        "commentCount": "12"
    },
    {
        "title": "Fundo Kinea Cr\u00e9dito Agro - FIAGRO (KNCA11) - Resumo Mensal ref. Julho.22",
        "viewCount": "38447",
        "likeCount": "84",
        "commentCount": "0"
    },
    {
        "title": "KNRI11 - Resumo Mensal do Kinea Renda ref. Julho.22",
        "viewCount": "27318",
        "likeCount": "465",
        "commentCount": "19"
    },
    {
        "title": "Fundos de A\u00e7\u00f5es Kinea - Resumo Mensal ref. Julho.22",
        "viewCount": "23433",
        "likeCount": "25",
        "commentCount": "0"
    },
    {
        "title": "KFOF11 - Resumo Mensal d

Transformando o JSON em Dataframe

In [16]:
df = pd.json_normalize(videos_df)
df

,title,viewCount,likeCount,commentCount
0,"Fundos de CRIs (KNCR11, KNIP11, KNHY11, KNSC11...",26780,247,9
1,Fundo de Infraestrutura KDIF11 - Resumo Mensal...,26060,254,12
2,Fundo Kinea Crédito Agro - FIAGRO (KNCA11) - R...,38447,84,0
3,KNRI11 - Resumo Mensal do Kinea Renda ref. Jul...,27318,465,19
4,Fundos de Ações Kinea - Resumo Mensal ref. Jul...,23433,25,0
...,...,...,...,...
289,Fundos de Ações Kinea - Resumo Mensal ref. Fev.20,255,17,0
290,KNRI11 - Resumo Mensal do Kinea Renda ref. Jan.20,481,44,0
291,Prev RF Kinea - Resumo Mensal ref. Jan20,113,3,0
292,Fundos de Ações Kinea - Resumo Mensal Ref. Jan.20,230,4,0


Renomeando as colunas

In [17]:
df.rename(columns={"title": "Título", "viewCount": "Visualizações", "likeCount": "Likes", "commentCount": "Comentários"}, inplace=True)
df

,Título,Visualizações,Likes,Comentários
0,"Fundos de CRIs (KNCR11, KNIP11, KNHY11, KNSC11...",26780,247,9
1,Fundo de Infraestrutura KDIF11 - Resumo Mensal...,26060,254,12
2,Fundo Kinea Crédito Agro - FIAGRO (KNCA11) - R...,38447,84,0
3,KNRI11 - Resumo Mensal do Kinea Renda ref. Jul...,27318,465,19
4,Fundos de Ações Kinea - Resumo Mensal ref. Jul...,23433,25,0
...,...,...,...,...
289,Fundos de Ações Kinea - Resumo Mensal ref. Fev.20,255,17,0
290,KNRI11 - Resumo Mensal do Kinea Renda ref. Jan.20,481,44,0
291,Prev RF Kinea - Resumo Mensal ref. Jan20,113,3,0
292,Fundos de Ações Kinea - Resumo Mensal Ref. Jan.20,230,4,0


Separando as datas de referência dos títulos

In [18]:
titles = df['Título']
df[['Título', 'Ref']] = df['Título'].str.rsplit("ef.", expand=True) # ver se tem como não bugar coluna de título
df['Título'] = titles

df

,Título,Visualizações,Likes,Comentários,Ref
0,"Fundos de CRIs (KNCR11, KNIP11, KNHY11, KNSC11...",26780,247,9,Julho.22
1,Fundo de Infraestrutura KDIF11 - Resumo Mensal...,26060,254,12,Julho.22
2,Fundo Kinea Crédito Agro - FIAGRO (KNCA11) - R...,38447,84,0,Julho.22
3,KNRI11 - Resumo Mensal do Kinea Renda ref. Jul...,27318,465,19,Julho.22
4,Fundos de Ações Kinea - Resumo Mensal ref. Jul...,23433,25,0,Julho.22
...,...,...,...,...,...
289,Fundos de Ações Kinea - Resumo Mensal ref. Fev.20,255,17,0,Fev.20
290,KNRI11 - Resumo Mensal do Kinea Renda ref. Jan.20,481,44,0,Jan.20
291,Prev RF Kinea - Resumo Mensal ref. Jan20,113,3,0,Jan20
292,Fundos de Ações Kinea - Resumo Mensal Ref. Jan.20,230,4,0,Jan.20


Separando por fundo

In [19]:
# lista de fundos retirados do site https://www.kinea.com.br/fundos/
funds = ['KNCR11', 'KNIP11', 'KNHY11', 'KNSC11', 'KCRE11', 'KDIF11', 'KEVE11', 'KNPR11', 'KFOF11', 'KNRI11', 'KNRE11', 'KINP11', 'KNCA11']

df['Fundos Citados'] = df['Título'].str.findall(f'({"|".join(funds)})').str.join(',')

df

,Título,Visualizações,Likes,Comentários,Ref,Fundos Citados
0,"Fundos de CRIs (KNCR11, KNIP11, KNHY11, KNSC11...",26780,247,9,Julho.22,"KNCR11,KNIP11,KNHY11,KNSC11,KCRE11"
1,Fundo de Infraestrutura KDIF11 - Resumo Mensal...,26060,254,12,Julho.22,KDIF11
2,Fundo Kinea Crédito Agro - FIAGRO (KNCA11) - R...,38447,84,0,Julho.22,KNCA11
3,KNRI11 - Resumo Mensal do Kinea Renda ref. Jul...,27318,465,19,Julho.22,KNRI11
4,Fundos de Ações Kinea - Resumo Mensal ref. Jul...,23433,25,0,Julho.22,
...,...,...,...,...,...,...
289,Fundos de Ações Kinea - Resumo Mensal ref. Fev.20,255,17,0,Fev.20,
290,KNRI11 - Resumo Mensal do Kinea Renda ref. Jan.20,481,44,0,Jan.20,KNRI11
291,Prev RF Kinea - Resumo Mensal ref. Jan20,113,3,0,Jan20,
292,Fundos de Ações Kinea - Resumo Mensal Ref. Jan.20,230,4,0,Jan.20,


In [13]:
df.to_csv("videos.csv", index = False)

Contando quantas vezes cada fundo é mencionado nos títulos

In [20]:
df['Fundos Citados'].str.split(',').explode('Fundos Citados').value_counts()

          137
KNRI11     30
KFOF11     30
KNCR11     28
KNIP11     28
KNHY11     28
KDIF11     28
KNSC11     20
KNCA11      7
KINP11      7
KEVE11      6
KCRE11      2
Name: Fundos Citados, dtype: int64